In [1]:
import os
from PIL import Image
import numpy as np

In [2]:
def read_image(im) :

    # Opening image as greyscale
    img = Image.open(im).convert('L')
    img.load()

    # Converting image into a numpy array
    data = np.asarray(img, dtype="int32")

    return data

In [3]:
trainFile = 'train_sample2.txt'
testFile = 'train_sample.txt'

In [14]:
metadata = {}
metadata['class2id'] = {}
metadata['im2label'] = []
metadata['img_dir'] = [] 
metadata['label2id'] = {}
ids = 0

# labels = {}
# images_labels = []
# images = []
# labelinv = {}

with open(trainFile, 'r') as f:
    for line in f:
        filePath, label = line.split(' ')
        if label not in metadata['class2id']:
            metadata['class2id'][label] = ids
            metadata['label2id'] = label
            ids = ids + 1
        metadata['im2label'].append(metadata['class2id'][label])
        metadata['img_dir'].append(filePath)

In [48]:
# def PCA_decompose(images, num_comp):
    
#     data = None
#     MSE = np.zeros(60)
#     for x in images:
#         img = read_image(x)
#         img = img.flatten()
#         if data is None:
#             data = img
#         else:
#             data = np.vstack([data, img])

#     # Find mean of the images
#     mean = np.mean(data.T, axis=1)
    
#     # Center the matrix by subtracting the mean
#     centered = data - mean
    
#     # Find covariance matrix
#     cov = np.matmul(centered, centered.T)

#     # Retrieving the eigenvalues and eigenvectors
#     values, vectors = np.linalg.eig(cov)
#     idx = values.argsort()[::-1]
#     values = np.real(values[idx])
#     vectors = np.real(vectors[:,idx])
    
#     # Finding projection matrix
#     projection = np.dot(vectors.T, centered)
    
#     eVecs = np.matmul(centered.T, vectors)
#     eVecsNorm = np.sqrt(np.sum(eVecs*eVecs, axis=0))
#     actualEVecs = np.real(eVecs/eVecsNorm)

#     return actualEVecs, np.real(centered), np.real(mean)

def pca(images):
    A = np.zeros((len(images),128*128*3),dtype='float64')
    for idx, img in enumerate(images):
        im = Image.open(img).resize((128, 128), Image.ANTIALIAS)
        im = np.array(im).reshape(-1)
        A[idx,:] = im/255.0
    A_mean = np.mean(A, axis=0)
    A = A - A_mean
    
    covM = np.matmul(A, A.T)
    # can also use eigh as it is faster and returns the eigenvalues in sorted order (it works only for symmetric matrices, A x A^T is symmetric)
    eVals, eVecs = np.linalg.eig(covM)
    idx = eVals.argsort()[::-1]
    eVals = np.real(eVals[idx])
    eVecs = np.real(eVecs[:,idx])
    
    eVecs = np.matmul(A.T, eVecs)
    eVecsNorm = np.sqrt(np.sum(eVecs*eVecs, axis=0))
    actualEVecs = eVecs/eVecsNorm
    
    return (actualEVecs, A, A_mean)

In [49]:
def find_probability(x):
    shiftx = x - np.max(x)
    exps = np.exp(shiftx)
    return exps / np.sum(exps, axis = 0)

In [50]:
vec, ce, mean = pca(metadata['img_dir'])
imgs = np.matmul(vec[:,0:32].T, ce.T)

In [51]:
y = np.array(metadata['im2label'])

In [54]:
W = np.random.rand(len(metadata['class2id']), 32)

eta = 0.005
numIterations = 60000

for i in range(0, numIterations):
    wtx_labels = find_probability(np.matmul(W, imgs))
    wtx_labels[y[:], np.arange(len(metadata['img_dir']))] += -1.0
    DSoftMax = (np.matmul(wtx_labels, imgs.T) / float(len(metadata['img_dir']))) + 0.0001 * W
    W = W - eta*(DSoftMax)


# labels = metadata['class2id']
# images_labels = metadata['im2label']
# images = metadata['img_dir']
# labelinv = metadata['label2id']

# W = np.random.rand(len(labels), 32)

# eta = 0.005
# numIterations = 60000

# for i in range(0, numIterations):
#     wtx_labels = find_probability(np.matmul(W, imgs))
#     wtx_labels[y[:], np.arange(len(images))] += -1.0
#     DSoftMax = (np.matmul(wtx_labels, imgs.T) / float(len(images))) + 0.0001 * W
#     W = W - eta*(DSoftMax)

In [57]:
correct = 0
total = 0

with open(testFile, 'r') as f:
    for x in f:
        x = x.strip()
        if x:
            filePath = x.split()[0]
            im = Image.open(filePath).resize((256, 256), Image.ANTIALIAS)
            im = np.array(im).reshape(-1)
#             im = im - mean
            im = np.matmul(vec[:,0:32].T, im).T
            print(metadata['label2id'][np.argmax(find_probability(np.matmul(W, im)))])
            total = total + 1
            if metadata['label2id'][np.argmax(find_probability(np.matmul(W, im)))] == filePath.split('/')[-1].split('_')[0]:
                correct = correct + 1

ValueError: shapes (32,49152) and (196608,) not aligned: 49152 (dim 1) != 196608 (dim 0)

In [58]:
import os
whole_data = os.listdir('./dataset')
total = 0
correct = 0

for file in whole_data:
    img = read_image('./dataset/' + file)
    img = img.flatten()
    pro_img = np.matmul(projection, img.T)
    print(metadata['label2id'][np.argmax(find_probability(np.matmul(W, im)))])

NameError: name 'projection' is not defined